In [ ]:
print("Hello world")

In [16]:
from openai import OpenAI

from dotenv import load_dotenv
load_dotenv()


client = OpenAI()
question = input("Enter your query: ")
prompt = f""" You are a data analyst.

There are 3 datasets and user will ask questions based on these datasets:
- sample_bo_tbl_large.csv: country (a 3 digit code like "AAA"), date, total_mins, international_mins, sms, total_data_usage, payg_amount
- sample_sub_details_large.csv: country, channel, date, subs, netadds, churn
- sample_revenue_large.csv: country, channel, date, revenue, net_revenue

Your task:
- Identify relevant datasets (with alias)
- Suggest joins (if needed): left, right, on, how
- Provide filters as list of {{"column": "...", "operator": "...", "value": ...}}
- Indicate aggregation type ("sum", "mean", "correlation", etc.)
- List involved columns

Your response should be only a JSON in the following format which will be an input as parameters to a method which performs these operations using pandas:
{{
  "datasets": ["name":..., "alias":...],
  "joins": [...],
  "filters": [...],
  "aggregation": "...",
  "columns": [...]
}}
If there is a date in the filters convert it into relevant dates of format dd-mm-yyyy and add it to the JSON.
Think step by step and then form the params JSON.
Double check all the values before giving the response. Donot confuse between operators == and =.
If the user asks queries that cannot be answered using the datasets, return a json in the following format:
{{
    "error":"the query cannot be answered using the available datasets"
}}
User Question: {question}"""
response = client.chat.completions.create(
    model = "gpt-4o",
    messages = [{"role":"user","content":prompt}],
)
print(response.choices[0].message.content)

```json
{
    "error":"the query cannot be answered using the available datasets"
}
```


In [17]:
from openai import OpenAI

from dotenv import load_dotenv
load_dotenv()


client = OpenAI()
question = input("Enter your query: ")
prompt = f""" You are a data analyst.

There are 3 datasets and user will ask questions based on these datasets:
- sample_bo_tbl_large.csv: country (a 3 digit code like "AAA"), date, total_mins, international_mins, sms, total_data_usage, payg_amount
- sample_sub_details_large.csv: country, channel, date, subs, netadds, churn
- sample_revenue_large.csv: country, channel, date, revenue, net_revenue

Your task is to generate an SQL query to answer the user question. Just return the SQL query.
Think step by step and then formulate the query.
If the query cannot be answered from the available datasets, just respond with "Query out of available data range."
User Question: {question}"""
response = client.chat.completions.create(
    model = "gpt-4o",
    messages = [{"role":"user","content":prompt}],
)
print(response.choices[0].message.content)

To answer the user question, you need to calculate the churn-to-subscriber ratio for each country in Q1 and then determine which country had the highest ratio.

To achieve this, follow these steps:
1. Filter the data for Q1 (January, February, March).
2. For each country, calculate the churn-to-subscriber ratio by dividing the sum of churn by the sum of subs.
3. Identify the country with the highest churn-to-subscriber ratio.

Here's the SQL query to perform these steps:

```sql
WITH Q1_data AS (
    SELECT 
        country, 
        SUM(churn) AS total_churn, 
        SUM(subs) AS total_subs
    FROM 
        sample_sub_details_large
    WHERE 
        EXTRACT(MONTH FROM TO_DATE(date, 'YYYY-MM-DD')) IN (1, 2, 3)
    GROUP BY 
        country
)
SELECT 
    country, 
    (total_churn::decimal / NULLIF(total_subs, 0)) AS churn_to_subs_ratio
FROM 
    Q1_data
ORDER BY 
    churn_to_subs_ratio DESC
LIMIT 1;
```

This query first filters out the relevant data for Q1, calculates the total ch

In [26]:
print(response.choices[0].message.content.split("```")[1])

sql
WITH Q1_data AS (
    SELECT 
        country, 
        SUM(churn) AS total_churn, 
        SUM(subs) AS total_subs
    FROM 
        sample_sub_details_large
    WHERE 
        EXTRACT(MONTH FROM TO_DATE(date, 'YYYY-MM-DD')) IN (1, 2, 3)
    GROUP BY 
        country
)
SELECT 
    country, 
    (total_churn::decimal / NULLIF(total_subs, 0)) AS churn_to_subs_ratio
FROM 
    Q1_data
ORDER BY 
    churn_to_subs_ratio DESC
LIMIT 1;



In [12]:
import os
print("Before loading .env:", os.getenv("OPENAI_API_KEY"))

Before loading .env: your_api_key_here
